In [3]:
import math
import random

In [4]:
# log loss because binary outputs
# sigmoid as activation funciton

In [5]:
def log_loss(y_hat, y):
    """
    y_hat is the prediction
    y is the true value
    """
    if y == 1:
        return -math.log(y_hat)
    elif y == 0:
        return -math.log(1-y_hat)
    else:
        raise InvalidInputError("y must be 0 or 1 but it is ", y)

In [6]:
def sigmoid(z):
    return 1 / (1 + (math.e**-z))

## Derivatives

In [7]:
def by_derivative(y_hat, y):
    """
    y bias derivative with respect to log loss
    """
    return (y_hat-y)

In [8]:
def Wy_derivative(y_hat, y, a):
    """
    y weights derivative with respect to log loss
    """
    return by_derivative(y_hat,y)*a

In [9]:
def ba_derivative(y_hat, y, Wy, an):
    """
    x bias derivative
    """
    return (y_hat-y)*Wy*an*(1-an)

In [10]:
def Waa_derivative(y_hat, y, Wy, an, a_prev):
    return ba_derivative(y_hat, y, Wy, an) * a_prev

def Wax_derivative(y_hat, y, Wy, an, x):
    return ba_derivative(y_hat, y, Wy, an) * x

## Sample data

In [13]:
X = []

for i in range(1000):
    sequence_length = random.randint(4,10)
    
    current_sequence = []
    for j in range(sequence_length):
        current_sequence.append(random.choice([0,1]))
        
    X.append(current_sequence)

In [17]:
y = []
for sequence in X:
    shifted_sequence = [0] + sequence[:-1]
    y.append(shifted_sequence)

In [14]:
X[:10]

[[0, 1, 1, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 1, 0, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [0, 0, 1, 1, 0, 1],
 [1, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0]]

In [18]:
y[:10]

[[0, 0, 1, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 0],
 [0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1]]

## Define the RNN

In [26]:
class RNN:
    def __init__(self):
        self.Wy = random.random()
        self.by = random.random()
        
        self.Waa = random.random()
        self.Wax = random.random()
        self.ba = random.random()